In [6]:
from airflow import DAG
from airflow.decorators import task
from airflow.models import Param
from airflow.providers.postgres.hooks.postgres import PostgresHook
from airflow.providers.sqlite.hooks.sqlite import SqliteHook
from airflow.providers.postgres.hooks.postgres import PostgresHook
import numpy as np
from airflow_clickhouse_plugin.hooks.clickhouse import ClickHouseHook
from pendulum import datetime
import requests
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

hook = ClickHouseHook(clickhouse_conn_id="conn_analytic_db")
hook.execute("show tables;")


[2025-04-19T10:41:38.004+0200] {base.py:84} INFO - Retrieving connection 'conn_analytic_db'
[2025-04-19T10:41:38.005+0200] {clickhouse.py:79} INFO - show tables;


[('dim_time',), ('fact_clicks',)]

In [ ]:
# from clickhouse_driver import Client

# client = Client(host='localhost', port=9000)
# result = client.execute('SELECT 1')

import clickhouse_connect

client = clickhouse_connect.get_client(host='localhost', port=8123)
client.query('SELECT 1')


In [ ]:
# Test Data Table Creation
table_ = """CREATE TABLE IF NOT EXISTS dim_time (
    timestamp TIMESTAMP, 
    year UInt32, 
    month UInt32, 
    day UInt32, 
    weekday UInt32, 
)
ENGINE = MergeTree()
PRIMARY KEY (timestamp)
"""
hook.execute(table_)

In [ ]:
hook.execute("SELECT * FROM analytic_db.fact_clicks")

In [7]:
# Test Insertion into Clickhouse
hook = PostgresHook("conn_production_db")
# Filter by Dates (daily?)
results = hook.get_records("SELECT " \
"c.advertiser_id, " \
"c.id campaign_id, " \
"ci.created_at " \
"FROM campaign c join " \
" clicks ci on c.id = ci.campaign_id")
results

hook = ClickHouseHook(clickhouse_conn_id="conn_analytic_db")
insert_query = f"INSERT INTO default.fact_clicks (advertiser_id, campaign_id, timestamp) VALUES"
chunks = np.array_split(results, 100)  # Split into 100 chunks
for chunk in chunks:
    values = [(row[0], row[1], row[2]) for row in chunk]
    result = hook.execute(insert_query, values)
logger.info(f"Inserted postgresql.{result} rows into fact_clicks table.")



[2025-04-19T10:41:41.201+0200] {base.py:84} INFO - Retrieving connection 'conn_production_db'
[2025-04-19T10:41:41.218+0200] {sql.py:624} INFO - Running statement: SELECT c.advertiser_id, c.id campaign_id, ci.created_at FROM campaign c join  clicks ci on c.id = ci.campaign_id, parameters: None
[2025-04-19T10:41:41.230+0200] {sql.py:633} INFO - Rows affected: 1200
[2025-04-19T10:41:41.238+0200] {base.py:84} INFO - Retrieving connection 'conn_analytic_db'
[2025-04-19T10:41:41.238+0200] {clickhouse.py:79} INFO - INSERT INTO default.fact_clicks (advertiser_id, campaign_id, timestamp) VALUES with [(5, 15, datetime.datetime(2025, 4, 12, 6, 4, 50, 14036)), (5, 15, datetime.datetime(2025, 4, 9, 7, 23, 41, 165176)), (5, 15, datetime.datetime(2025, 4, 10, 23, 17, 42, 262153)), (5, 15, datetime.datetime(2025, 4, 10, 7, 40, 39, 288976)), (5, 15, datetime.datetime(2025, 4, 11, 3, 38, 54, 258502)), (5, 15, datetime.datetime(2025, 4, 11, 15, 24, 10, 578021)), (5, 15, datetime.datetime(2025, 4, 12, 11